In [45]:
import os

In [49]:
%pwd

'/Users/kausik'

In [50]:
%cd '/Users/kausik/Desktop/IK/End_to_end_MLOPS_project'


/Users/kausik/Desktop/IK/End_to_end_MLOPS_project


In [35]:
os.chdir("../")

In [51]:
%pwd

'/Users/kausik/Desktop/IK/End_to_end_MLOPS_project'

In [52]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/chat.kausik/End_to_end_mlops.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="chat.kausik"
os.environ["MLFLOW_TRACKING_PASSWORD"]="69d6ab76cdc60dcca2812e53bad1012a97bd4c18"

In [53]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [54]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [55]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            # mlflow_uri="https://dagshub.com/someshnaman/End_to_end_MLOPS_project.mlflow",
            mlflow_uri="https://dagshub.com/chat.kausik/End_to_end_mlops.mlflow",
           
        )

        return model_evaluation_config


In [56]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [57]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
                print("AJ")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [60]:
import mlflow
mlflow.set_tracking_uri("http://localhost:8050")

In [61]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-26 21:06:44,622: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-26 21:06:44,624: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-26 21:06:44,626: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-26 21:06:44,626: INFO: common: created directory at: artifacts]
[2025-01-26 21:06:44,627: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-26 21:06:44,636: WARNING: connectionpool: Retrying (Retry(total=4, connect=4, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x138d30460>: Failed to establish a new connection: [Errno 61] Connection refused')': /api/2.0/mlflow/runs/create]
[2025-01-26 21:06:48,641: WARNING: connectionpool: Retrying (Retry(total=3, connect=3, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x138d306a0>: Failed to establish a new c

MlflowException: API request to http://localhost:8050/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='localhost', port=8050): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x138d30d60>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [44]:
print("Kausik Chattapadhyay")

Kausik Chattapadhyay
